## Projeto de Consulta Juridica

In [1]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

#servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 103
Current browser version is 105.0.5195.102 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00895FD3+2187219]
	Ordinal0 [0x0082E6D1+1763025]
	Ordinal0 [0x00743E78+802424]
	Ordinal0 [0x007651F9+938489]
	Ordinal0 [0x00760DDC+921052]
	Ordinal0 [0x0075E691+910993]
	Ordinal0 [0x007917F0+1120240]
	Ordinal0 [0x0079144A+1119306]
	Ordinal0 [0x0078C9A6+1100198]
	Ordinal0 [0x00766F80+946048]
	Ordinal0 [0x00767E76+949878]
	GetHandleVerifier [0x00B390C2+2721218]
	GetHandleVerifier [0x00B2AAF0+2662384]
	GetHandleVerifier [0x0092137A+526458]
	GetHandleVerifier [0x00920416+522518]
	Ordinal0 [0x00834EAB+1789611]
	Ordinal0 [0x008397A8+1808296]
	Ordinal0 [0x00839895+1808533]
	Ordinal0 [0x008426C1+1844929]
	BaseThreadInitThunk [0x7701FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77167A7E+286]
	RtlGetAppContainerNamedObjectPath [0x

In [ ]:
# abrir a página index (entrar no site da busca jurídica)
import os
caminho = os.getcwd()
arquivo = caminho + r"\index.html"

###### Importar a base de dados

In [ ]:
import pandas as pd

tabela = pd.read_excel('Processos.xlsx')
display(tabela)

In [ ]:
from operator import index
from selenium.webdriver import ActionChains
import time 
from cgitb import text

for linha in tabela.index:
  # para cada processo (linha da tabela)
  navegador.get(arquivo)

  #abrir lista de cidades
  botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
  ActionChains(navegador).move_to_element(botao).perform()

  cidade = tabela.loc[linha, 'Cidade']
  print(cidade)

  # Selecionando o Rio de Janeiro
  navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

  # mudar para a nova aba
  aba_original = navegador.window_handles[0]
  indice = 1 + linha
  nova_aba = navegador.window_handles[indice]

  navegador.switch_to.window(nova_aba)
  # preencher o formulário com os dados de busca
  navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, "Nome"])
  navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, "Advogado"])
  navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, "Processo"])

  # clicar em pesquisar
  navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
  
  # confirmar a pesquisa
  alerta = navegador.switch_to.alert
  alerta.accept()

  # esperar o resultado da pesquisa e agir de acordo com o resultado
  
  while True:
    try:
      alerta = navegador.switch_to.alert
      break
    except:
      time.sleep(1) 
  texto_alerta = alerta.text

  if 'Processo encontrado com sucesso' in texto_alerta:
    alerta.accept()
    tabela.loc[linha, "Status"] = "Encontrado"
  else:
    tabela.loc[linha, "Status"] = "Não encontrado"
    alerta.accept()

In [ ]:
navegador.quit()
display(tabela)

tabela.to_excel("Processo Atualizado.xlsx")